In [ ]:
!pip install tensorflow==2.4.0 tfx==0.27.0 tensorflow_addons==0.12.1 fsspec==0.8.7 keras-tuner

# Download and Prepare the data

In [1]:
# import keras
from tensorflow import keras  

# Download the dataset and split into train and test sets 
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [2]:
# Normalise the pixels between 0 and 1 
img_train = img_train.astype('float32')/255.0
img_test = img_test.astype('float32')/255.0


# Baseline Performance

In [3]:
input_ = keras.layers.Input(shape=(28,28))
flatten_ = keras.layers.Flatten()(input_)
layer_ = keras.layers.Dense(512, 'relu', name='dense_1')(flatten_)
dropout_ = keras.layers.Dropout(0.2)(layer_)
dense_ = keras.layers.Dense(10, activation='softmax')(dropout_)
b_model = keras.Model(input_, dense_)
b_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
# Setup the training parameters
b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy'])

In [5]:
# Number of training epochs.
NUM_EPOCHS = 10

# Train the model
b_model.fit(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6510 - accuracy: 0.7705 - val_loss: 0.3958 - val_accuracy: 0.8570
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4033 - accuracy: 0.8542 - val_loss: 0.3649 - val_accuracy: 0.8687
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3602 - accuracy: 0.8684 - val_loss: 0.3782 - val_accuracy: 0.8580
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3364 - accuracy: 0.8758 - val_loss: 0.3429 - val_accuracy: 0.8730
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3130 - accuracy: 0.8835 - val_loss: 0.3266 - val_accuracy: 0.8811
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.2979 - accuracy: 0.8877 - val_loss: 0.3579 - val_accuracy: 0.8688
Epoch 7/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2921 - accuracy: 0.8904 - val_loss: 0.3392 - val_accuracy:

In [6]:
# Evaluate model on the test set
b_eval_dict = b_model.evaluate(img_test, label_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3463 - accuracy: 0.8811


In [7]:
# Define helper function
def print_results(model, model_name, eval_dict):
  '''
  Prints the values of the hyparameters to tune, and the results of model evaluation

  Args:
    model (Model) - Keras model to evaluate
    model_name (string) - arbitrary string to be used in identifying the model
    eval_dict (dict) -  results of model.evaluate
  '''
  print(f'\n{model_name}:')

  print(f'number of units in 1st Dense layer: {model.get_layer("dense_1").units}')
  print(f'learning rate for the optimizer: {model.optimizer.lr.numpy()}')

  for key,value in eval_dict.items():
    print(f'{key}: {value}')

# Print results for baseline model
print_results(b_model, 'BASELINE MODEL', b_eval_dict)


BASELINE MODEL:
number of units in 1st Dense layer: 512
learning rate for the optimizer: 0.0010000000474974513
loss: 0.3463492691516876
accuracy: 0.8810999989509583


# Keras Tuner

To perform hypertuning with Keras Tuner, you will need to:

* Define the model
* Select which hyperparameters to tune
* Define its search space
* Define the search strategy

## Define the model

The model you set up for hypertuning is called a *hypermodel*. When you build this model, you define the hyperparameter search space in addition to the model architecture. 

You can define a hypermodel through two approaches:

* By using a model builder function
* By [subclassing the `HyperModel` class](https://keras-team.github.io/keras-tuner/#you-can-use-a-hypermodel-subclass-instead-of-a-model-building-function) of the Keras Tuner API


In this lab, you will take the first approach: you will use a model builder function to define the image classification model. This function returns a compiled model and uses hyperparameters you define inline to hypertune the model. 

The function below basically builds the same model you used earlier. The difference is there are two hyperparameters that are setup for tuning:

* the number of hidden units of the first Dense layer
* the learning rate of the Adam optimizer

You will see that this is done with a HyperParameters object which configures the hyperparameter you'd like to tune. For this exercise, you will: 

* use its `Int()` method to define the search space for the Dense units. This allows you to set a minimum and maximum value, as well as the step size when incrementing between these values. 

* use its `Choice()` method for the learning rate. This allows you to define discrete values to include in the search space when hypertuning.

You can view all available methods and its sample usage in the [official documentation](https://keras-team.github.io/keras-tuner/documentation/hyperparameters/#hyperparameters).

In [9]:
# Import required packages
import tensorflow as tf 
import kerastuner as kt  

def model_builder(hp):
  '''
  Builds the model and sets up the hyperparameters to tune.

  Args:
    hp - Keras tuner object

  Returns:
    model with hyperparameters to tune
  '''
  # Initialize the Model API 
  input_ = keras.layers.Input(shape=(28,28))
  flatten_ = keras.layers.Flatten()(input_)

  # Tune the number of layers in the first Dense layer
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  layer_ = keras.layers.Dense(hp_units, 'relu', name='dense_1')(flatten_)
  dropout_ = keras.layers.Dropout(0.2)(layer_)
  dense_ = keras.layers.Dense(10, activation='softmax')(dropout_)
  b_model = keras.Model(input_, dense_)

  # Tune the learning rate for the optimizer
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  b_model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

  return b_model

## Instantiate the Tuner and perform hypertuning

Now that you have the model builder, you can then define how the tuner can find the optimal set of hyperparameters, also called the search strategy. Keras Tuner has [four tuners](https://keras-team.github.io/keras-tuner/documentation/tuners/) available with built-in strategies - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. 

In this tutorial, you will use the Hyperband tuner. Hyperband is an algorithm specifically developed for hyperparameter optimization. It uses adaptive resource allocation and early-stopping to quickly converge on a high-performing model. This is done using a sports championship style bracket wherein the algorithm trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. You can read about the intuition behind the algorithm in section 3 of [this paper](https://arxiv.org/pdf/1603.06560.pdf).

Hyperband determines the number of models to train in a bracket by computing 1 + log<sub>`factor`</sub>(`max_epochs`) and rounding it up to the nearest integer. You will see these parameters (i.e. `factor` and `max_epochs` passed into the initializer below). In addition, you will also need to define the following to instantiate the Hyperband tuner:

* the hypermodel (built by your model builder function)
* the `objective` to optimize (e.g. validation accuracy)
* a `directory` to save logs and checkpoints for every trial (model configuration) run during the hyperparameter search. If you re-run the hyperparameter search, the Keras Tuner uses the existing state from these logs to resume the search. To disable this behavior, pass an additional `overwrite=True` argument while instantiating the tuner.
* the `project_name` to differentiate with other runs. This will be used as a subdirectory name under the `directory`.

You can refer to the [documentation](https://keras.io/api/keras_tuner/tuners/hyperband/) for other arguments you can pass in.

In [10]:
# Instantiate the tuner
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='kt_dir',
                     project_name='kt_hyperband')

In [11]:
# Display hypertuning settings
tuner.search_space_summary()

In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [14]:
# Perform hypertuning
NUM_EPOCHS = 50
tuner.search(img_train, label_train, epochs=NUM_EPOCHS, validation_split=0.2, callbacks=[stop_early])

Epoch 1/2
1500/1500 [==============================] - 5s 3ms/step - loss: 0.7739 - accuracy: 0.7319 - val_loss: 0.5011 - val_accuracy: 0.8188
Epoch 2/2
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5610 - accuracy: 0.7979 - val_loss: 0.4947 - val_accuracy: 0.8226


Epoch 1/2
1500/1500 [==============================] - 5s 3ms/step - loss: 0.6927 - accuracy: 0.7586 - val_loss: 0.4255 - val_accuracy: 0.8480
Epoch 2/2
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4146 - accuracy: 0.8509 - val_loss: 0.3772 - val_accuracy: 0.8667


Epoch 1/2
1500/1500 [==============================] - 5s 3ms/step - loss: 1.0434 - accuracy: 0.6556 - val_loss: 0.5313 - val_accuracy: 0.8249
Epoch 2/2
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5275 - accuracy: 0.8229 - val_loss: 0.4616 - val_accuracy: 0.8423


Epoch 1/2
1500/1500 [==============================] - 7s 4ms/step - loss: 0.6426 - accuracy: 0.7736 - val_loss: 0.3932 - val_accuracy: 0.8593
Epoch 2/2
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3981 - accuracy: 0.8543 - val_loss: 0.3828 - val_accuracy: 0.8636


Epoch 1/2
1500/1500 [==============================] - 8s 5ms/step - loss: 0.6446 - accuracy: 0.7692 - val_loss: 0.4820 - val_accuracy: 0.8148
Epoch 2/2
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3959 - accuracy: 0.8561 - val_loss: 0.3671 - val_accuracy: 0.8664


...

Epoch 1/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.7594 - accuracy: 0.7340 - val_loss: 0.5470 - val_accuracy: 0.8138
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5734 - accuracy: 0.7974 - val_loss: 0.4842 - val_accuracy: 0.8315
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5400 - accuracy: 0.8082 - val_loss: 0.4696 - val_accuracy: 0.8277
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5241 - accuracy: 0.8122 - val_loss: 0.4421 - val_accuracy: 0.8347
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5079 - accuracy: 0.8183 - val_loss: 0.4937 - val_accuracy: 0.8164
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5069 - accuracy: 0.8188 - val_loss: 0.4678 - val_accuracy: 0.8258
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5081 - accuracy: 0.8154 - val_loss: 0.4700 - val_accuracy:

Epoch 1/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.7032 - accuracy: 0.7534 - val_loss: 0.4241 - val_accuracy: 0.8488
Epoch 2/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.4217 - accuracy: 0.8470 - val_loss: 0.3802 - val_accuracy: 0.8583
Epoch 3/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3744 - accuracy: 0.8627 - val_loss: 0.3517 - val_accuracy: 0.8744
Epoch 4/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3547 - accuracy: 0.8709 - val_loss: 0.3437 - val_accuracy: 0.8755
Epoch 5/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3248 - accuracy: 0.8775 - val_loss: 0.3348 - val_accuracy: 0.8807
Epoch 6/10
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3212 - accuracy: 0.8812 - val_loss: 0.3302 - val_accuracy: 0.8823
Epoch 7/10
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3071 - accuracy: 0.8848 - val_loss: 0.3376 - val_accuracy:

Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.9937 - accuracy: 0.6776 - val_loss: 0.5137 - val_accuracy: 0.8249
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5090 - accuracy: 0.8264 - val_loss: 0.4492 - val_accuracy: 0.8470
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4508 - accuracy: 0.8470 - val_loss: 0.4192 - val_accuracy: 0.8530
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4110 - accuracy: 0.8581 - val_loss: 0.3938 - val_accuracy: 0.8631
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3886 - accuracy: 0.8649 - val_loss: 0.3811 - val_accuracy: 0.8644
Epoch 6/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3697 - accuracy: 0.8707 - val_loss: 0.3657 - val_accuracy: 0.8717
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3593 - accuracy: 0.8754 - val_loss: 0.3624 - val_accuracy:

Epoch 1/10
1500/1500 [==============================] - 7s 4ms/step - loss: 1.0268 - accuracy: 0.6618 - val_loss: 0.5146 - val_accuracy: 0.8261
Epoch 2/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5133 - accuracy: 0.8269 - val_loss: 0.4553 - val_accuracy: 0.8422
Epoch 3/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4570 - accuracy: 0.8442 - val_loss: 0.4225 - val_accuracy: 0.8528
Epoch 4/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4258 - accuracy: 0.8544 - val_loss: 0.4007 - val_accuracy: 0.8602
Epoch 5/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3962 - accuracy: 0.8612 - val_loss: 0.3830 - val_accuracy: 0.8668
Epoch 6/10
1500/1500 [==============================] - 7s 4ms/step - loss: 0.3767 - accuracy: 0.8676 - val_loss: 0.3806 - val_accuracy: 0.8683
Epoch 7/10
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3566 - accuracy: 0.8738 - val_loss: 0.3642 - val_accuracy:

INFO:tensorflow:Oracle triggered exit


In [15]:
# Get the optimal hyperparameters from the results
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 160 and the optimal learning rate for the optimizer
is 0.001.



# Build and Train the model

In [16]:
# Build the model with the optimal hyperparameters
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 160)               125600    
_________________________________________________________________
dropout (Dropout)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1610      
Total params: 127,210
Trainable params: 127,210
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Train the hypertuned model
h_model.fit(img_train, label_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6944 - accuracy: 0.7564 - val_loss: 0.4161 - val_accuracy: 0.8504
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4123 - accuracy: 0.8528 - val_loss: 0.3848 - val_accuracy: 0.8588
Epoch 3/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3827 - accuracy: 0.8604 - val_loss: 0.3641 - val_accuracy: 0.8675
Epoch 4/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3522 - accuracy: 0.8704 - val_loss: 0.3697 - val_accuracy: 0.8575
Epoch 5/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3309 - accuracy: 0.8763 - val_loss: 0.3465 - val_accuracy: 0.8766
Epoch 6/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3124 - accuracy: 0.8836 - val_loss: 0.3279 - val_accuracy: 0.8844
Epoch 7/10
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3070 - accuracy: 0.8849 - val_loss: 0.3408 - val_accuracy:

In [18]:
# Evaluate the hypertuned model against the test set
h_eval_dict = h_model.evaluate(img_test, label_test, return_dict=True)

313/313 [==============================] - 1s 2ms/step - loss: 0.3597 - accuracy: 0.8758


In [19]:
# Print results of the baseline and hypertuned model
print_results(b_model, 'BASELINE MODEL', b_eval_dict)
print_results(h_model, 'HYPERTUNED MODEL', h_eval_dict)


BASELINE MODEL:
number of units in 1st Dense layer: 512
learning rate for the optimizer: 0.0010000000474974513
loss: 0.3463492691516876
accuracy: 0.8810999989509583

HYPERTUNED MODEL:
number of units in 1st Dense layer: 160
learning rate for the optimizer: 0.0010000000474974513
loss: 0.35974106192588806
accuracy: 0.8758000135421753
